In [1]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from statistics import mean
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
## Image Transforms
train_transform = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.RandomRotation(30),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])  


test_transform = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

## Loading train and test data using Image Folder
base_path = r"N:\00.Datasets\Cat_Dog_data"
train_data = datasets.ImageFolder(os.path.join(base_path,"train"), transform= train_transform)
test_data = datasets.ImageFolder(os.path.join(base_path,"test"), transform= test_transform )

## Data Loader
trainloader = DataLoader(train_data, batch_size=64, shuffle=True)
testloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [2]:
## Loading pre trained model
model = models.densenet121(pretrained = True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
## Setting req_grad = Fals
for param in model.features.parameters():
    param.requires_grad = False

print(list(model.features.parameters())[0].requires_grad)
list(model.classifier.parameters())[0].requires_grad

In [ ]:
class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


model.classifier = SmallNet()

model

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.003)

In [ ]:
epochs = 1
print_every = 40
final_trl = []
final_tel =[]
final_accuracy = []


for e in range(epochs):
    running_loss = 0
    total=0
    correct=0
    train_losses = []
    test_losses = []
    epoch_accurracy =[]
    print(f"Epoch: {e+1}/{epochs}")

    for i, (images, labels) in enumerate(iter(trainloader)):


        
        optimizer.zero_grad()
        
        logits = model.forward(images)   # 1) Forward pass
        pred = F.log_softmax(logits, dim=1)
        loss = criterion(pred, labels) # 2) Compute loss
        loss.backward()                  # 3) Backward pass
        optimizer.step()                 # 4) Update model
        
        running_loss += loss.item()

        train_losses.append(loss.item())
        
        
        if i % print_every == 0:
            print(f"\tIteration: {i}\t Loss: {running_loss/print_every:.4f}")
            running_loss = 0

    
    model.eval()
    with T.no_grad():
        for i, (images, labels) in enumerate(iter(testloader)):

            logits = model.forward(images)
            test_pred = F.log_softmax(logits, dim=1)

            test_loss = criterion(test_pred, labels)

            test_losses.append(test_loss.item())

            _,predicted = test_pred.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            accuracy = 100*correct/total


            epoch_accurracy.append(accuracy)


            



    model.train()
    
    mean_trl = mean(train_losses)
    mean_tel = mean(test_losses)
    mean_acc = mean(epoch_accurracy)
    final_trl.append(mean_trl)
    final_tel.append(mean_tel)
    final_accuracy.append(mean_acc)
    
    if (e+1) % 1 == 0:
        T.save(model.state_dict(), f'checkpoint_{e+1}.pth')

    #accuracies.append(acc)
    print(f'Epoch: {e + 1} | loss: {mean_trl} | test loss: {mean_tel} | accuracy:{mean_acc} ')